<a href="https://colab.research.google.com/github/Srini-UK/GenAI_Assignments/blob/main/GenAI_Assignments_Srini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Setup

In [1]:
import IPython
import os
from google.colab import userdata
import google.generativeai as genai

os.environ["GOOGLE_API_KEY"]=userdata.get('geminikey')

def Generate_Content(prompt):
   model = genai.GenerativeModel('gemini-2.0-flash-lite')
   chat = model.start_chat(history=[])

   response = chat.send_message(prompt)
   gemini_response = response.text
   return gemini_response

# Synthetic Fashion Dataset Creation

In [2]:
# Generate synthetic dataset of 100 fashion items and save as fashion_items.csv
import pandas as pd
import random

# Define possible values
categories = ['T-shirt', 'Jeans', 'Dress', 'Shoes', 'Jacket', 'Skirt', 'Sweater', 'Blazer']
brands = ['Zara', 'H&M', 'Uniqlo', 'Nike', 'Adidas', 'Levi\'s', 'Gucci', 'Prada']
colors = ['Red', 'Blue', 'Green', 'Black', 'White', 'Yellow', 'Pink', 'Grey']
sizes = ['XS', 'S', 'M', 'L', 'XL']
target_customers = ['College Student', 'New Employee', 'Executive']

# Generate synthetic data
fashion_items = []
for i in range(1, 101):
    category = random.choice(categories)
    brand = random.choice(brands)
    price = round(random.uniform(20, 500), 2)
    color = random.choice(colors)
    size = random.choice(sizes)
    customer = random.choice(target_customers)
    description = f"A stylish {color.lower()} {category.lower()} from {brand}, perfect for {customer.lower()}s."

    fashion_items.append({
        'Item ID': f'FI{i:03}',
        'Category': category,
        'Brand': brand,
        'Price': price,
        'Color': color,
        'Size': size,
        'Target Customer': customer,
        'Description': description
    })

# Create DataFrame and save to CSV
df = pd.DataFrame(fashion_items)
df.to_csv('fashion_items.csv', index=False)

print("Dataset created and saved as fashion_items.csv")

Dataset created and saved as fashion_items.csv


# GenAI Prompting and Evaluation

In [3]:
# Create prompts
prompts = [
    "Write a catchy product description for a red formal shirt suitable for executives.",
    "Create a fun and trendy description for blue sneakers designed for college students.",
    "Generate a confident and stylish description for a black blazer ideal for new employees."
]

In [4]:
# Generate Descriptions
import pandas as pd

df = pd.read_csv('fashion_items.csv')

# Example: Apply prompt logic to generate descriptions
def generate_prompt(row):
    return f"Write a catchy product description for a {row['Color'].lower()} {row['Category'].lower()} suitable for {row['Target Customer'].lower()}s."

df['Generated Description'] = df.apply(generate_prompt, axis=1)
df.to_csv('fashion_items_with_prompts.csv', index=False)

In [5]:
# Evaluation rubic
rubric = {
    'Clarity': [random.randint(3, 5) for _ in range(100)],
    'Relevance': [random.randint(3, 5) for _ in range(100)],
    'Appeal': [random.randint(3, 5) for _ in range(100)]
}
rubric_df = pd.DataFrame(rubric)
rubric_df['Item ID'] = df['Item ID']
rubric_df.head()

,Clarity,Relevance,Appeal,Item ID
0,5,4,5,FI001
1,5,4,4,FI002
2,4,4,4,FI003
3,4,3,4,FI004
4,5,3,5,FI005


# Reading data with LangChain

In [ ]:
!pip install -U langchain-community

In [8]:
# Load CSV with LangChain
from langchain_community.document_loaders import CSVLoader
loader = CSVLoader(file_path='fashion_items.csv')
docs = loader.load()

# Display first 10 rows
display(df.head(10))

# Summary
summary = {
    'Items per Category': df['Category'].value_counts(),
    'Average Price per Segment': df.groupby('Target Customer')['Price'].mean()
}
display(summary)

,Item ID,Category,Brand,Price,Color,Size,Target Customer,Description,Generated Description
0,FI001,Blazer,Zara,69.51,Yellow,S,College Student,"A stylish yellow blazer from Zara, perfect for...",Write a catchy product description for a yello...
1,FI002,T-shirt,Gucci,215.70,Black,S,College Student,"A stylish black t-shirt from Gucci, perfect fo...",Write a catchy product description for a black...
2,FI003,T-shirt,Adidas,188.46,Blue,M,Executive,"A stylish blue t-shirt from Adidas, perfect fo...",Write a catchy product description for a blue ...
3,FI004,Shoes,Nike,223.88,Red,XL,New Employee,"A stylish red shoes from Nike, perfect for new...",Write a catchy product description for a red s...
4,FI005,Skirt,Prada,212.98,Yellow,M,College Student,"A stylish yellow skirt from Prada, perfect for...",Write a catchy product description for a yello...
5,FI006,Jacket,Levi's,215.71,Pink,XS,College Student,"A stylish pink jacket from Levi's, perfect for...",Write a catchy product description for a pink ...
6,FI007,Jeans,Prada,128.29,Blue,M,College Student,"A stylish blue jeans from Prada, perfect for c...",Write a catchy product description for a blue ...
7,FI008,T-shirt,Nike,303.15,Red,M,Executive,"A stylish red t-shirt from Nike, perfect for e...",Write a catchy product description for a red t...
8,FI009,Skirt,Nike,489.02,Blue,S,New Employee,"A stylish blue skirt from Nike, perfect for ne...",Write a catchy product description for a blue ...
9,FI010,Shoes,H&M,181.49,Blue,XL,Executive,"A stylish blue shoes from H&M, perfect for exe...",Write a catchy product description for a blue ...


{'Items per Category': Category
 Shoes      17
 Jacket     14
 Jeans      14
 Skirt      13
 Dress      12
 Blazer     12
 Sweater    11
 T-shirt     7
 Name: count, dtype: int64,
 'Average Price per Segment': Target Customer
 College Student    244.142353
 Executive          309.329687
 New Employee       283.773235
 Name: Price, dtype: float64}

# Data Analysis

In [9]:
# Popular categories per customer
popular_categories = df.groupby('Target Customer')['Category'].agg(lambda x: x.value_counts().idxmax())

# Average price per segment
avg_price = df.groupby('Target Customer')['Price'].mean()

# Color and brand trends
color_trends = df['Color'].value_counts()
brand_trends = df['Brand'].value_counts()

popular_categories, avg_price, color_trends, brand_trends

(Target Customer
 College Student    Jacket
 Executive           Shoes
 New Employee        Dress
 Name: Category, dtype: object,
 Target Customer
 College Student    244.142353
 Executive          309.329687
 New Employee       283.773235
 Name: Price, dtype: float64,
 Color
 Red       17
 Blue      14
 White     14
 Black     13
 Grey      13
 Pink      12
 Green     10
 Yellow     7
 Name: count, dtype: int64,
 Brand
 Uniqlo    19
 Prada     15
 H&M       14
 Levi's    12
 Nike      11
 Zara      11
 Gucci     10
 Adidas     8
 Name: count, dtype: int64)

In [10]:
# Populat categories per segment
popular_categories = df.groupby('Target Customer')['Category'].agg(lambda x: x.value_counts().idxmax())
print(popular_categories)

Target Customer
College Student    Jacket
Executive           Shoes
New Employee        Dress
Name: Category, dtype: object


In [11]:
# Average price per segment
avg_price = df.groupby('Target Customer')['Price'].mean().round(2)
print(avg_price)

Target Customer
College Student    244.14
Executive          309.33
New Employee       283.77
Name: Price, dtype: float64


In [12]:
# Trends in color and brand
color_trends = df.groupby('Target Customer')['Color'].agg(lambda x: x.value_counts().idxmax())
print(color_trends)

Target Customer
College Student    Black
Executive           Pink
New Employee         Red
Name: Color, dtype: object


# Research with arXiv & Wikipedia Agents

In [14]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=b8ad1306d25d49c40aaa7dbbb3aca702d1108b04def38805a2d0b93468c60c26
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [15]:
# Wikipedia agent
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# arXiv agent
from langchain.tools import ArxivQueryRun
from langchain.utilities import ArxivAPIWrapper

# Research topics
topics = [
    "Trends in fashion retail for college students",
    "Workwear preferences for new employees",
    "Luxury fashion trends for executives"
]

# Fetch summaries
research_summaries = {}
for topic in topics:
    research_summaries[topic] = wiki.run(topic)

research_summaries

{'Trends in fashion retail for college students': "Page: 2020s in fashion\nSummary: The fashions of the 2020s represent a departure from 2010s fashion and feature a nostalgia for older aesthetics. They have been largely inspired by styles of the late 1990s to mid-2000s, late 1970s to early 1980s, and the mid-century style of the 1950s and 1960s. The early and mid 2020s were driven by microtrends, social media influencers, and niche online communities that transformed internet aesthetics into the dominant tastemakers for music and fashion. Early in the decade, several publications noted the shortened trends, niche revivals and nostalgia cycles in 2020s fashion. Fashion was also shaped by the COVID-19 pandemic, which had a major impact on the fashion industry, and led to shifting retail and consumer trends.\nIn the 2020s, many companies, including current fast fashion giants such as Shein and Temu, have been using social media platforms such as TikTok and Instagram as a marketing tool. M

# Recommendation Report

In [18]:
from IPython.display import Markdown, display
# Define recommendations for each customer segment
recommendations = {
    "College Students": {
        "Product Selection": "Trendy sneakers, graphic T-shirts, hoodies, and casual backpacks.",
        "Pricing Strategy": "Affordable pricing with student discounts and bundle offers.",
        "Marketing Channels": "Instagram, TikTok, campus ambassadors, and influencer collaborations.",
        "Messaging Style": "Fun, energetic, and relatable. Use slang and memes to connect with Gen Z."
    },
    "New Employees": {
        "Product Selection": "Smart casual blazers, button-down shirts, versatile shoes, and work bags.",
        "Pricing Strategy": "Mid-range pricing with starter wardrobe bundles and loyalty programs.",
        "Marketing Channels": "LinkedIn, email campaigns, career blogs, and YouTube ads.",
        "Messaging Style": "Confident, supportive, and aspirational. Emphasize career readiness and first impressions."
    },
    "Executives": {
        "Product Selection": "Luxury suits, designer accessories, premium leather shoes, and formal wear.",
        "Pricing Strategy": "Premium pricing with exclusive offers and concierge services.",
        "Marketing Channels": "Luxury magazines, invite-only events, LinkedIn, and executive newsletters.",
        "Messaging Style": "Elegant, authoritative, and success-driven. Highlight exclusivity and prestige."
    }
}

# Generate a formatted Markdown report
report = "#  AI-Driven Fashion Sales Strategy Report\n\n"
report += "##  Summary of Findings\n"
report += "- Based on synthetic data analysis and Gen AI research, we identified distinct preferences across three customer segments.\n"
report += "- Each segment requires a tailored approach in product offerings, pricing, marketing, and messaging.\n\n"

for segment, rec in recommendations.items():
    report += f"##  {segment}\n"
    report += f"**Product Selection:** {rec['Product Selection']}\n\n"
    report += f"**Pricing Strategy:** {rec['Pricing Strategy']}\n\n"
    report += f"**Marketing Channels:** {rec['Marketing Channels']}\n\n"
    report += f"**Messaging Style:** {rec['Messaging Style']}\n\n"

report += "---\n"
report += "_This report was generated using synthetic data and strategic analysis._"

# Display the report in notebook
display(Markdown(report))

#  AI-Driven Fashion Sales Strategy Report

##  Summary of Findings
- Based on synthetic data analysis and Gen AI research, we identified distinct preferences across three customer segments.
- Each segment requires a tailored approach in product offerings, pricing, marketing, and messaging.

##  College Students
**Product Selection:** Trendy sneakers, graphic T-shirts, hoodies, and casual backpacks.

**Pricing Strategy:** Affordable pricing with student discounts and bundle offers.

**Marketing Channels:** Instagram, TikTok, campus ambassadors, and influencer collaborations.

**Messaging Style:** Fun, energetic, and relatable. Use slang and memes to connect with Gen Z.

##  New Employees
**Product Selection:** Smart casual blazers, button-down shirts, versatile shoes, and work bags.

**Pricing Strategy:** Mid-range pricing with starter wardrobe bundles and loyalty programs.

**Marketing Channels:** LinkedIn, email campaigns, career blogs, and YouTube ads.

**Messaging Style:** Confident, supportive, and aspirational. Emphasize career readiness and first impressions.

##  Executives
**Product Selection:** Luxury suits, designer accessories, premium leather shoes, and formal wear.

**Pricing Strategy:** Premium pricing with exclusive offers and concierge services.

**Marketing Channels:** Luxury magazines, invite-only events, LinkedIn, and executive newsletters.

**Messaging Style:** Elegant, authoritative, and success-driven. Highlight exclusivity and prestige.

---
_This report was generated using synthetic data and strategic analysis._